In [ ]:
#ARQUIVO PARA PESQUISA DE ID

import pandas as pd
from datetime import datetime, timedelta

print("Carregando e preparando os dados para a demonstração. Aguarde...")
try:
    df = pd.read_csv('df_t_cidades.csv', sep=',')
    df['date_purchase'] = pd.to_datetime(df['date_purchase'])
    df = df.sort_values(by=['fk_contact', 'date_purchase'])
    print(">>> Dados prontos para consulta! <<<")
except FileNotFoundError:
    print("ERRO: Arquivo 'df_t_cidades.csv' não foi encontrado. Encerrando o programa.")
    exit()


def demonstrar_para_cliente(client_id, dataframe):
    """
    Recebe um ID de cliente, calcula todas as métricas e exibe um
    relatório formatado na tela.
    """
    print("\n" + "="*50)
    print(f"Gerando relatório para o cliente: {client_id}")
    print("="*50)

    historico_cliente = dataframe[dataframe['fk_contact'] == client_id]

    if historico_cliente.empty:
        print(f"ATENÇÃO: Cliente com ID '{client_id}' não encontrado.")
        print("="*50)
        return

    total_compras = len(historico_cliente)
    ultima_compra = historico_cliente['date_purchase'].max()
    destino_provavel = historico_cliente['destination_city_departure'].mode().iloc[0] if not historico_cliente['destination_city_departure'].mode().empty else "N/D"

    print(f"  > Total de Compras no Histórico: {total_compras}")
    print(f"  > Última Compra Registrada:       {ultima_compra.strftime('%d/%m/%Y')}")
    print(f"  > Destino Mais Provável:          {destino_provavel}")

    if total_compras > 1:
        frequencia = historico_cliente['date_purchase'].diff().mean()
        frequencia_dias = frequencia.days

        hoje = pd.to_datetime(datetime.now())

        proxima_data = ultima_compra + frequencia

        if proxima_data < hoje:
            while proxima_data < hoje:
                proxima_data += frequencia

        limite_30_dias = hoje + timedelta(days=30)
        viaja_em_30 = 'Sim' if hoje <= proxima_data <= limite_30_dias else 'Não'

        print(f"  > Frequência Média de Compra:     A cada {frequencia_dias:.0f} dias")
        print("-" * 50)
        print("  PREDIÇÕES:")
        print(f"  > Próxima Compra Provável em:     {proxima_data.strftime('%d/%m/%Y')}")
        print(f"  > Comprará nos Próximos 30 Dias?  {viaja_em_30}")

    else:
        print("-" * 50)
        print("  PREDIÇÕES:")
        print("  > Cliente com apenas 1 compra. Não é possível calcular a frequência ou prever a próxima data.")

    print("="*50)


while True:
    id_desejado = input("\nDigite o ID do cliente para demonstrar (ou 'sair' para encerrar): ")

    if id_desejado.lower() in ['sair', 'exit', 'quit']:
        print("\nEncerrando a demonstração. Até logo!")
        break

    demonstrar_para_cliente(id_desejado, df)